# 东方财富网国内新闻数据抓取器
## 拂晓工作室
### 此程序将爬取某个板块所有最近的新闻并存进CSV！

# 参考资料
* [1] BeautifulSoup的例子：https://qiita.com/rusarusa/items/d7f014ba80d6fe7a3e07
* [2] BeautifulSoup的文档：https://www.cnblogs.com/zhaof/p/6930955.html
* [3] How to install BS4 aggressively: https://github.com/ipython/ipython/issues/10684
* [4] CSV文档：https://docs.python.org/2/library/csv.html
* [5] CSV使用实例：https://blog.csdn.net/waple_0820/article/details/70049953

In [5]:
# 导入库
import urllib
import re
import pandas as pd
import pymysql
import os
import turicreate
import csv
from bs4 import BeautifulSoup


In [6]:
# 新闻板块
china = 'cgnjj'  # 国内
international = 'cgjjj'  # 国际

# 定义基本常量
url = 'http://finance.eastmoney.com/news/' + china + '.html' # 主目录地址 更换不同主目录以爬取不同板块内容 (BAD)
csv_filepath = '../Datasets/Eastmoney/news/'  # 存储数据文件地址

In [7]:
# 创建一个二十五页的list
page_list = []
counter = 1

while counter <= 25:
    pageurl = 'http://finance.eastmoney.com/news/' + china
    if counter != 1:
        pageurl = pageurl + '_' + str(counter) + '.html'
        page_list.append(pageurl)
    else:
        pageurl = pageurl + '.html'
        page_list.append(pageurl)
    counter += 1
    
print('成功创建包含 ' + str(len(page_list)) + ' 个页面链接的目录！')
print(page_list)

成功创建包含 25 个页面链接的目录！
['http://finance.eastmoney.com/news/cgnjj.html', 'http://finance.eastmoney.com/news/cgnjj_2.html', 'http://finance.eastmoney.com/news/cgnjj_3.html', 'http://finance.eastmoney.com/news/cgnjj_4.html', 'http://finance.eastmoney.com/news/cgnjj_5.html', 'http://finance.eastmoney.com/news/cgnjj_6.html', 'http://finance.eastmoney.com/news/cgnjj_7.html', 'http://finance.eastmoney.com/news/cgnjj_8.html', 'http://finance.eastmoney.com/news/cgnjj_9.html', 'http://finance.eastmoney.com/news/cgnjj_10.html', 'http://finance.eastmoney.com/news/cgnjj_11.html', 'http://finance.eastmoney.com/news/cgnjj_12.html', 'http://finance.eastmoney.com/news/cgnjj_13.html', 'http://finance.eastmoney.com/news/cgnjj_14.html', 'http://finance.eastmoney.com/news/cgnjj_15.html', 'http://finance.eastmoney.com/news/cgnjj_16.html', 'http://finance.eastmoney.com/news/cgnjj_17.html', 'http://finance.eastmoney.com/news/cgnjj_18.html', 'http://finance.eastmoney.com/news/cgnjj_19.html', 'http://finance.eastm

In [8]:
# 创建一个包含20个class的list
counter = 0
class_list = []

while counter < 20:
    class_list.append('newsTr' + str(counter))
    counter += 1
    
print('成功创建包含 ' + str(len(class_list)) + ' 个class的目录！')
print(class_list)  # Debug

成功创建包含 20 个class的目录！
['newsTr0', 'newsTr1', 'newsTr2', 'newsTr3', 'newsTr4', 'newsTr5', 'newsTr6', 'newsTr7', 'newsTr8', 'newsTr9', 'newsTr10', 'newsTr11', 'newsTr12', 'newsTr13', 'newsTr14', 'newsTr15', 'newsTr16', 'newsTr17', 'newsTr18', 'newsTr19']


In [9]:
# 创造一个CSV文件以及基本文件结构 [5]
with open("../DataSets/Eastmoney/news/Datarecent_news.csv", "w") as csvfile: 
    writer = csv.writer(csvfile)

    # 先写入columns_name
    writer.writerow(["index", "date", "title", "contents", "related_stocks"])
    # 写入多行用writerows
    # writer.writerows([[0,1,3],[1,2,3],[2,3,4]])

In [ ]:
# 抓取新闻URL函数
for each_page in page_list:
    for each_class in class_list:
        print('hey')  # Placeholder



